## CSE 255 Programming Assignment 1: Collinear points

For this programming assignment, we'll be using a Jupyter notebook.

### In this programming assignment, you will write python3 code using pyspark to find sets of collinear points given arbitrary number of 2D points.

Definition of collinearity[1]: In geometry, collinearity of a set of points is the property of their lying on a single line. A set of points with this property is said to be collinear.

![](non-collinear-points.jpg)

Here, points P,Q,R and A,R,B are collinear. However, points A,B,C are non-collinear. For more, refer [2].

Given an input file with a set of co-ordinates, your task is to use pyspark library functions to write a program to find if three or more points are collinear.

For instance, if given these points: {(1,1), (0,1), (2,2), (3,3), (0,5), (3,4), (5,6), (0,-3), (-2,-2)}

Sets of collinear points are: {((-2,-2), (1,1), (2,2), (3,3)), ((0,1), (3,4), (5,6)), ((0,-3), (0,1), (0,5))}. Note that the ordering of the points in a set or the order of the sets does not matter. 

Note: Every set of collinear points has to have atleast three points (single point or a pair of points always lie on a line).

1. https://en.wikipedia.org/wiki/Collinearity
2. http://www.mathcaptain.com/geometry/collinear-points.html

### Initialize spark context using 4 local cores as workers.
Note that we can create a SparkConf() object and use it to initialize the spark context. 

In [ ]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Collinear Points").setMaster("local[4]")
sc = SparkContext(conf=conf)

from pyspark.rdd import RDD

### Guidelines for implementation

The goal of this assignment is to make you familiar with programming using pyspark. There are many ways to find sets of collinear points from a list of points. For the purposes of this assignment, we shall stick with the below approach:

1. Find the cartesian product of the list of points with itself. For example, given three points [(1,0), (2,0), (3,0)], we first construct [((1,0), (2,0)), ((2,0), (1,0)), ((1,0), (3,0)), ((3,0), (1,0)), ((2,0), (3,0)), ((3,0), (2,0))]. Note that pairs ((1,0),(1,0)), ((2,0),(2,0)) and ((3,0),(3,0)) are deliberately not included in the result of cartesian product, since they all have duplicate points.

2. Use the above intermediary result to find slope (of the line segment connecting) each pair of points in the cartesian product. If two pairs of points have the same slope, and if they have one point in common, the three points have to be collinear. For example, in [((1,0), (2,0)), ((2,0), (1,0)), ((1,0), (3,0)), ((3,0), (1,0)), ((2,0), (3,0)), ((3,0), (2,0))], slope between (1,0) and (2,0) is 0, and slope between (1,0) and (3,0) is also 0. Since (1,0) is common to both of these pairs of points, ((1,0), (2,0), (3,0)) must be collinear.


Keeping the above technique in mind, you will complete the leftover parts of this notebook. You are required to use pyspark RDD API's map, reduce, groupby and other library functions to do so. Refer: http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=rdd#pyspark.RDD

Besides the visible assert statements in this notebook, your implementations will be tested against hidden tests after the assignment submission deadline.

*NOTE*: some of the cells in this notebook are locked i.e. you will not be able to modify them. This is intended to guide you in implementation, and for grading purposes. In case you want to split a locked cell to see intermediate results, you may copy the contents of the locked cell to a new cell and do your testing there. You *should* remove these new cells before submitting your notebook.  

### Below are some helper functions that will be used in your implementations. You are neither required to nor encouraged to change the definitions of these functions.

In [ ]:
def format_result(x):
    """
    input: ((A,slope), [C1,..., Ck]) where each of A, C1,..., Ck is a point of form (Ax, Ay) and slope is a float.
    output: (C1,..., Ck, A)
    
    Concatenates collinear points.
    """
    x[1].append(x[0][0])
    return tuple(x[1])

def to_sorted_points(x):
    """
    Sorts and returns a tuple of points for further processing.
    """
    return tuple(sorted(x))

### Below are some functions that you will implement. You should follow the function definitions, and use them appropriately elsewhere in the notebook.

In [ ]:
def to_tuple(x):
    """
    Converts each point of form 'Ax, Ay' into a point of form (Ax, Ay) for further processing.
    Example: '2,3' is converted to (2,3)
    """
    # YOUR CODE HERE
    raise NotImplementedError()

def non_duplicates(x):
    """  
    input: Pair (A,B) where A and B are of form (Ax, Ay) and (Bx, By) respectively.
    Returns True if A != B, False otherwise.
    
    Use this function inside the get_cartesian() function to 'filter' out pairs with duplicate points. 
    """
    # YOUR CODE HERE
    raise NotImplementedError()

def get_cartesian(rdd):
    """
    Does a Cartesian product of an RDD with itself and returns an RDD with DISTINCT pairs of points.
    
    Refer:  http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=cartesian#pyspark.RDD.cartesian
    """
    
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
## Let's test the to_tuple(x) implementation (1 Point)
assert to_tuple('1 1') == (1,1)
assert type(to_tuple('1 1')) == tuple
assert type(to_tuple('1 1')[0])==int and type(to_tuple('1 1')[1])==int

In [ ]:
## Let's test the non_duplicates(x) implementation (1 Point)
assert non_duplicates(((0,0),(1,2))) == True
assert non_duplicates(((0,0),(0,0))) == False

In [ ]:
## Let's test the get_cartesian(x) implementation (3 Points)

test_rdd = sc.parallelize([(1,0), (2,0), (3,0)])
assert get_cartesian(test_rdd).collect() == [((1, 0), (2, 0)), ((1, 0), (3, 0)), ((2, 0), (1, 0)), ((2, 0), (3, 0)), ((3, 0), (1, 0)), ((3, 0), (2, 0))]
assert isinstance(get_cartesian(test_rdd), RDD) == True



In [ ]:

def find_slope(x):
    """
    Computes slope between points A and B.
    
    input: Pair (A,B) where A and B are of form (Ax, Ay) and (Bx, By) respectively. Example input: ((1,2),(3,4))
    output: Pair ((A,slope), B). Where, A and B have the same definition as input, 
            and slope refers to the slope of the line segment connecting point A and B. Example output: (((1,2),1),(3,4))
    
    NOTE: If Ax == Bx, use slope as "inf".
    """
    # YOUR CODE HERE
    raise NotImplementedError()



def find_collinear(rdd):
    
    """
    Finds collinear points.
    
    Input is an RDD (which is the output of the get_cartesian() function. 
    
    Output is an RDD which does the below operations:
    
    1. Find the slope of the line between all pairs of points A = (Ax, Ay) and B = (Bx, By).
    2. For each (A, B), find all points C = ((C1x, C1y), (C2x, C2y), ... (Cnx, Cny)) 
       where slope of (A,B) = slope of (A, Ci).
    3. Return (A, B, Ck) where Ck = all points of C which satisfy the condition 1.
    
    For an example, refer the assert statement test for this function.
    
    HINT:   You should use the above helper functions in conjunction with Spark RDD API (refer http://spark.apache.org/docs/latest/api/python/pyspark.html?highlight=rdd#pyspark.RDD)
            Finally, use helper function format_result() appropriately from inside this function after you have implemented the above operations.
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
## Let's test the find_slope(x) implementation (3 Points)

assert find_slope(((1,2),(3,4))) == (((1,2),1),(3,4))
assert find_slope(((1,2),(1,5))) == (((1,2),"inf"),(1,5))
assert find_slope(((1,2),(2,5))) == (((1,2),3),(2,5))
assert find_slope(((1,2),(-7,-2)))[0][1] == 0.5


In [ ]:
## Let's test the find_collinear(rdd) implementation (5 Points)

test_rdd = sc.parallelize([((4, 2), (2, 1)), ((4, 2), (-3, 4)), ((4, 2), (6, 3)), ((2, 1), (4, 2)), ((2, 1), (-3, 4)), ((2, 1), (6, 3)), ((-3, 4), (4, 2)), ((-3, 4), (2, 1)), ((-3, 4), (6, 3)), ((6, 3), (4, 2)), ((6, 3), (2, 1)), ((6, 3), (-3, 4))])
assert find_collinear(test_rdd).collect() == [((4, 2), (6, 3), (2, 1)), ((2, 1), (6, 3), (4, 2)), ((4, 2), (2, 1), (6, 3))]
assert isinstance(find_collinear(test_rdd), RDD) == True



In [ ]:
## Let's test find_collinear(rdd) when run with output of get_cartesian(rdd) (2 Points)

test_rdd = sc.parallelize([(4, -2), (2, -1), (-3,4), (6,3), (-9,4), (6, -3), (8,-4), (6,9)])
test_rdd = get_cartesian(test_rdd)
assert find_collinear(test_rdd).collect() == [((6, -3), (6, 9), (6, 3)), ((4, -2), (2, -1), (8, -4), (6, -3)), ((2, -1), (6, -3), (8, -4), (4, -2)), ((6, 3), (6, 9), (6, -3)), ((4, -2), (6, -3), (8, -4), (2, -1)), ((6, 3), (6, -3), (6, 9)), ((4, -2), (2, -1), (6, -3), (8, -4))]


### The cell below runs all of the above functions step by step. You are again neither required to nor encouraged to change the definitions of these functions.

In [ ]:
def get_collinear(filename):
    """
    Computes the set of collinear points
    """
    
    # Load the data file into an RDD
    rdd = sc.textFile(filename)
    
    # Transform the loaded points into tuples
    rdd = rdd.map(to_tuple)
    
    # Transform the RDD to now have the cartesian product of the RDD with itself
    rdd = get_cartesian(rdd)
    
    # Transform the RDD to now just have sets of Collinear Points 
    rdd = find_collinear(rdd)
    
    # Sorting each of your returned sets of collinear points. This is for grading purposes. You may ignore this.
    rdd = rdd.map(to_sorted_points)
    
    # Collecting the collinear points RDD in a set to remove duplicate sets of collinear points. This is for grading purposes. You may ignore this.
    res = set(rdd.collect())
    
    return res

In [ ]:
"""
    Check that the get_collinear() function returns the correct output for several input files.
    You may assume that input files do not have duplicate points. (10 points)
    
"""
assert get_collinear("data.txt") == {((-2, -2), (1, 1), (2, 2), (3, 3)), ((0, 1), (3, 4), (5, 6)), ((0, -3), (0, 1), (0, 5))}
assert get_collinear("data50.txt") == {((3, 6), (7, 4), (9, 3)), ((1, 6), (3, 6), (4, 6), (7, 6)), 
                                 ((0, 2), (3, 1), (6, 0)), ((1, 0), (2, 0), (5, 0), (6, 0)), 
                                 ((1, 3), (3, 6), (5, 9)), ((0, 8), (4, 6), (6, 5)), 
                                 ((6, 0), (6, 1), (6, 5), (6, 9)), 
                                 ((7, 2), (7, 3), (7, 4), (7, 6), (7, 8)), ((3, 1), (3, 3), (3, 6)), 
                                 ((0, 2), (1, 2), (5, 2), (7, 2)), ((0, 3), (2, 5), (3, 6), (6, 9)), 
                                 ((0, 2), (1, 3), (2, 4), (4, 6), (5, 7)), ((1, 2), (4, 3), (7, 4)), 
                                 ((0, 3), (4, 6), (8, 9)), ((9, 3), (9, 4), (9, 5)), ((2, 5), (5, 7), (8, 9)), 
                                 ((0, 5), (2, 4), (4, 3), (8, 1)), ((0, 8), (1, 6), (2, 4)), 
                                 ((3, 6), (5, 2), (6, 0)), ((5, 9), (6, 9), (8, 9)), 
                                 ((0, 8), (1, 8), (7, 8)), ((0, 4), (1, 3), (3, 1)), ((5, 9), (7, 6), (9, 3)), 
                                 ((1, 2), (2, 4), (3, 6)), ((0, 7), (1, 5), (3, 1)), 
                                 ((1, 5), (2, 4), (3, 3), (6, 0)), ((0, 2), (3, 3), (9, 5)), 
                                 ((0, 7), (1, 6), (2, 5), (4, 3), (5, 2), (6, 1)), 
                                 ((0, 4), (1, 5), (5, 9)), ((1, 5), (3, 6), (5, 7), (7, 8)), 
                                 ((1, 6), (3, 3), (5, 0)), ((3, 6), (4, 3), (5, 0)), 
                                 ((1, 2), (4, 5), (7, 8), (8, 9)), ((0, 2), (1, 1), (2, 0)), 
                                 ((3, 3), (4, 5), (5, 7), (6, 9)), ((0, 2), (0, 3), (0, 4), (0, 5), (0, 7), (0, 8)), 
                                 ((2, 0), (4, 3), (8, 9)), ((5, 7), (6, 5), (7, 3), (8, 1)), ((5, 0), (7, 6), (8, 9)), 
                                 ((5, 0), (6, 1), (7, 2), (9, 4)), ((0, 4), (1, 2), (2, 0)), 
                                 ((1, 1), (3, 1), (6, 1), (8, 1)), ((5, 7), (7, 6), (9, 5)), ((1, 1), (7, 4), (9, 5)), 
                                 ((0, 4), (2, 4), (7, 4), (9, 4)), ((1, 0), (3, 1), (5, 2), (7, 3), (9, 4)), 
                                 ((2, 0), (3, 3), (4, 6), (5, 9)), ((4, 3), (4, 5), (4, 6)), 
                                 ((1, 0), (4, 3), (6, 5), (7, 6)), ((0, 3), (2, 4), (4, 5)), 
                                 ((1, 6), (4, 5), (7, 4)), ((1, 0), (1, 1), (1, 2), (1, 3), (1, 5), (1, 6), (1, 8)), 
                                 ((0, 3), (1, 3), (3, 3), (4, 3), (7, 3), (9, 3)), ((0, 4), (2, 5), (4, 6)), 
                                 ((0, 7), (3, 6), (6, 5), (9, 4)), ((1, 8), (4, 6), (7, 4)), 
                                 ((0, 5), (3, 3), (6, 1)), ((1, 8), (3, 6), (4, 5), (7, 2), (8, 1)), 
                                 ((1, 2), (3, 1), (5, 0)), ((1, 1), (5, 2), (9, 3)), 
                                 ((5, 0), (5, 2), (5, 7), (5, 9)), ((0, 5), (1, 5), (2, 5), (4, 5), (6, 5), (9, 5)), 
                                 ((3, 1), (4, 5), (5, 9)), ((2, 0), (2, 4), (2, 5)), ((5, 2), (6, 5), (7, 8))}



In [ ]:
## TODO: Not sure we need this cell in the released HW. If not, please delete it.

## 0 points
"""Check that the get_collinear() function raises an error for invalid input files"""
try:
    get_collinear("data.txt")
except ValueError:
    raise AssertionError("Data points do not match the expected format.")
else:
    pass